# RAS-Commander (Parallel Execute)
Author: William (Bill) Katzenmeyer, P.E., C.F.M. (C.H. Fenstermaker and Associates, LLC)

Source: https://github.com/billk-FM/HEC-Commander-Tools

Last Update: 2023-11-11 Update .prj read function to allow for use with steady flow models

In [ ]:
#   --  INPUT HEC-RAS PROJECT FOLDER  --
# This script runs all HEC-RAS plans in parallel

# HEC-RAS Project Template Folder
HECRAS_Project_To_Run = r"Your_HECRAS_Project_with_Plans"

# Define Remote Target Folders
HECRAS_Deploy_Targets = [ 
r"C:\Local_Temp_Folder",
r"\\NetworkName1\RemoteTempFolder",
r"\\NetworkName2\RemoteTempFolder",
]  
# Accepts any combination of local and remote paths for deployment

# Define number of copies in each target (usually 2-4)
Number_Parallel_Runs = 3

# Optional step to rename DSS Outputs (Useful if using permutation plans in HEC-RAS)
Rename_DSS_Outputs = "No"  # "Yes" or "No"

# Define substring start and end positions for the new DSS file name
substring_start_position = 4
substring_end_position = 7
# For Example, a plan name with "AEP 100YR" would have it's DSS output renamed to "100YR.dss" with the above substring positions 

In [ ]:
# Additional Settings, Paths and Variables
# NOTE: Keep the outputfolder name the same as the template, there is a safety mechanism that looks at the first leters of the folder name

# ***********        FOR DEPLOY SCRIPT: HEC-RAS Version, Local Paths and File Type Exclusions           ***********

# Specify HEC-RAS EXE Path for .bat file creation (Change Version here if needed)
hecras_exe_path = r"C:\Program Files (x86)\HEC\HEC-RAS\6.2\Ras.exe"
print ("HEC-RAS Executable Path Used for batch file creation: ", hecras_exe_path) 

'''
** PSEXEC Control Options **
PSEXEC can be used to control number of cores, but requires each available core to be specified for each run
Not implemented here, but can be added by end users if needed.  Recommend using RAS option to limit # of cores instead
'''

# Flag to run PSEXEC in system account (Yes or No) (uses -s flag instead of -i {Psexec_Session_ID})
Psexec_Run_In_System_Account = "No"  # No is recommended, system account can hang without user feedback

# If not in system account, specify session ID to run in (usually 2 for single user machines) (-i {Psexec_Session_ID})
# Task Manager detail tab can be used to find Session ID on remote machine
Psexec_Session_ID = 2 

# Remote Deployment Task Priority (low, below normal, normal)
Psexec_Priority = "low" # low recommended for maximum responsiveness on multi-user machines

# Local Target Path used in Batch File Creation 
Remote_Share_Path = "C:\\"

# Build Remote_Base_Directory for batch file creation
Remote_Base_Directory = Remote_Share_Path + HECRAS_Deploy_Targets[0].split('\\', 3)[-1]
print ("Local Target Path Used for batch file creation: ", Remote_Base_Directory) 

# Folder Folder_Safety_Prefix:  This prevents the script from deleting unrelated folders 
# Specify how many of the folder prefix characters need to match  
Folder_Safety_Prefix_Length = 2

# OPTIONAL Prefix for filtering DSS files in HMS Folder (default is blank, returns all files)
# Useful if you have DSS files for multiple events in the same folder
HECHMS_dss_filter_prefix = ""

# Define exclusions for folder deployment 
# NOTE: .tif, .tiff are needed for 2D preprocessing
# For 1D, add these back for faster deployment: '*.tif', '*.tiff', '*.img',

exclusions = [ '*.p**.hdf']
print ("The following file types are not needed for 1D compute runs and will be excluded: ", exclusions)

# Number of Parallel Threads for File Copy Operations
File_Copy_Threads = 2
print ("Number of Parallel Threads for File Copy Operations: ", File_Copy_Threads)

In [ ]:
# Check for psexec64.exe, and download if missing
import urllib.request
import zipfile
import os
from urllib.error import URLError, HTTPError
import tempfile

# Get the temp directory path
temp_directory = tempfile.gettempdir()

# Specify the psexec path within this directory
psexec_path = os.path.join(temp_directory, "psexec", "PsExec64.exe")
print("Target PSEXEC Path:", psexec_path)

def download_and_extract_pstools(psexec_path):
    # Check if the directory exists
    dir_path = os.path.dirname(psexec_path)
    if not os.path.exists(dir_path):
        print(f"Directory {dir_path} does not exist.")
    else:
        # Check if PsExec64.exe already exists (case-insensitive)
        if any(os.path.exists(os.path.join(dir_path, f)) for f in os.listdir(dir_path) if f.lower() == 'psexec64.exe'):
            print(f"The file PsExec64.exe exists at {dir_path}.")
            return
    
    print(f"PsExec64.exe not found at {dir_path}. Proceeding to download and extract PSTools.")
    
    # Define the download URL
    download_url = 'https://download.sysinternals.com/files/PSTools.zip'
    
    # Specify the directory where all tools will be extracted
    destination_dir = dir_path
    
    # Create the destination directory if it doesn't exist
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)
        print(f"Created directory {destination_dir}.")
    
    # Define the path where the ZIP file will be saved
    zip_file_path = os.path.join(destination_dir, 'PSTools.zip')
    
    try:
        # Download the ZIP file
        urllib.request.urlretrieve(download_url, zip_file_path)
        print(f"Downloaded PSTools.zip to {zip_file_path}")
        
        # Extract all the contents of the ZIP file
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(destination_dir)
            print(f"Extracted all contents of PSTools.zip to {destination_dir}")
    
    except (HTTPError, URLError) as e:
        print(f"[ERROR] Could not download PSTools.zip due to {e}.")
        print("Please download PsExec manually from https://learn.microsoft.com/en-us/sysinternals/downloads/psexec")


# Download and extract PSTools if PsExec64.exe doesn't exist
download_and_extract_pstools(psexec_path)

In [ ]:
# Required Import Statements
# Auto-Install packages using subprocess (available in most base environments)
import sys
import subprocess 

packages = ["os", "shutil", "re", "pandas", "keyring", "getpass", "chardet", "datetime", "fnmatch", "threading", "time", "concurrent", "tqdm"]

# Logic to Install Packages
def install(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print("Installed " + package + " successfully")
    except subprocess.CalledProcessError:
        try:
            subprocess.check_call(["conda", "install", "-y", package])
        except subprocess.CalledProcessError as ex:
            print(f"Unable to install {package}: {str(ex)}")

for package in packages:
    try:
        # If the import succeeds, the package is installed, so we can move on
        __import__(package)
    except ImportError:
        # If the import fails, the package is not installed and we try to install it
        install(package)

import sys
import os
import shutil
import pandas as pd
import re
import chardet
import datetime
import fnmatch
import getpass
import keyring
import threading
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from queue import Queue, Empty
from tqdm import tqdm
import glob

In [ ]:
# Derive the HEC-RAS Project Name from the HEC-RAS Template Folder
def get_HECRAS_project_name(folder_path):
    # Scanning the RAS_FullyAuto_Folder for .rasmap files
    for file_name in os.listdir(folder_path):
        # Check if the file has .rasmap extension
        if file_name.endswith('.rasmap'):
            # Return the file name without the .rasmap extension
            return file_name[:-7]  
    # Return None if no .rasmap file is found
    return None

# Get the HEC-RAS Project Name 
HECRAS_project_name = get_HECRAS_project_name(HECRAS_Project_To_Run)
print("HEC-RAS Project Name:", HECRAS_project_name)

# HEC-RAS PRJ File is the project name with a .prj extension
HECRAS_prj_file = HECRAS_project_name + ".prj"
print("HEC-RAS PRJ File:", HECRAS_prj_file)

# Set Output CSV with Missing DSS files and Extension Numbers
plan_title_csv_file_name = "RAS-Cmdr New Plans and Titles.csv"
print("Plan Title Output CSV:", plan_title_csv_file_name)

Remote_Base_Directory = Remote_Share_Path + HECRAS_Deploy_Targets[0].split('\\', 3)[-1]
print ("Local Target Path Used for batch file creation: ", Remote_Base_Directory) 

In [ ]:
# Read PRJ and extract .pXX file mapping
 
# Read the .prj file
prj_path = os.path.join(HECRAS_Project_To_Run, HECRAS_prj_file)
with open(prj_path, "r") as prj_file:
    prj_content = prj_file.read()
 
# Read prj_content and use .pXX file to generate paths
def extract_plan_files(prj_content, output_folder):
    """Extract .pXX and their paths from .prj content and output folder."""
    # Extract .pXX references
    pXX_references = re.findall(r"Plan File=(p\d+)", prj_content)
   
    # Construct the paths based on the provided output folder
    data = []
    for pXX in pXX_references:
        pXX_path = os.path.join(output_folder, f"{HECRAS_project_name}.{pXX}")
        data.append((pXX, pXX_path))
 
    return pd.DataFrame(data, columns=['pXX', '.pXX Paths'])
 
# Call the function and store the result in the specified DataFrame
project_plan_unsteady_mapping_df = extract_plan_files(prj_content, HECRAS_Project_To_Run)
 
# Show the DataFrame
display(project_plan_unsteady_mapping_df)

In [ ]:
# Optional step to rename DSS Outputs (Useful if using permutation plans in HEC-RAS)

# Initialize DataFrame
Plan_Identifier_Df = pd.DataFrame(columns=['File Name', 'Short Identifier', 'Substring'])

# Define a prefix for the new DSS file line
prefix = "Harvey_2017_RAS_RoughnessCal_"

def get_pxx_files(directory_path):
    os.chdir(directory_path)
    pxx_files = glob.glob("WH*.p[0-9][0-9]")
    return pd.DataFrame(pxx_files, columns=['File Name'])

def get_short_identifier(df, index):
    file_path = df.at[index, 'File Name']
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "Short Identifier=" in line:
                short_identifier = line.split("Short Identifier=")[1].strip()
                df.at[index, 'Short Identifier'] = short_identifier

def get_substring(df, index, start_pos, end_pos):
    short_identifier = df.at[index, 'Short Identifier']
    substring = short_identifier[start_pos:end_pos]
    df.at[index, 'Substring'] = substring

def replace_dss_line(df, index, prefix):
    file_path = df.at[index, 'File Name']
    substring = df.at[index, 'Substring']
    print("substring = ", substring)
    with open(file_path, 'r') as file:
        lines = file.readlines()
    with open(file_path, 'w') as file:
        for line in lines:
            if "DSS File=" in line:
                new_line = f"DSS File=.\\{prefix}{substring}.dss\n"
                print(new_line)
                file.write(new_line)
            else:
                file.write(line)

# Main Script Logic
if Rename_DSS_Outputs == "Yes":
    Plan_Identifier_Df = get_pxx_files(HECRAS_Project_To_Run)
    
    for index in Plan_Identifier_Df.index:
        get_short_identifier(Plan_Identifier_Df, index)
        get_substring(Plan_Identifier_Df, index, substring_start_position, substring_end_position)
        print("Short Identifier for: ", Plan_Identifier_Df.at[index, 'File Name'], " is: ", Plan_Identifier_Df.at[index, 'Short Identifier'])   
        replace_dss_line(Plan_Identifier_Df, index, prefix)
        print("Renamed DSS File in: ", Plan_Identifier_Df.at[index, 'File Name'])

In [ ]:
#Locate HEC-RAS Project Name and Safety Prefix

def get_hecras_project_name(folder_path):
    # Scanning the HECRAS_Project_To_Run for .rasmap files
    for file_name in os.listdir(folder_path):
        # Check if the file has .rasmap extension
        if file_name.endswith('.rasmap'):
            # Return the file name without the .rasmap extension
            return file_name[:-7]  
    # Return None if no .rasmap file is found
    return None

def get_Folder_Safety_Prefix(folder_path, prefix_length):
    # Split the path by backslashes and get the last part (folder name)
    folder_name = folder_path.split("\\")[-1]
    # Return the specified number of starting characters from the folder name
    return folder_name[:prefix_length]

# Get the HEC-RAS Project Name 
hecras_project_name = get_hecras_project_name(HECRAS_Project_To_Run)
print("HEC-RAS Project Name:", hecras_project_name)

# HEC-RAS PRJ File is the project name with a .prj extension
ras_prj_file = hecras_project_name + ".prj"
print("HEC-RAS PRJ File:", ras_prj_file)

# Determine Folder_Safety_Prefix based on Folder_Safety_Prefix_Length
Folder_Safety_Prefix = get_Folder_Safety_Prefix(HECRAS_Project_To_Run, Folder_Safety_Prefix_Length)
print("Folder Safety Prefix:", Folder_Safety_Prefix)

In [ ]:
# Utilize Keyring to request and validate User Credentials for Remote Execution
# This should be your FULL Microsoft Windows user login (with domain or local machine name: DOMAIN\USER or MACHINE\USER)


# Constants
USERNAME_FILE = "username.txt"
SERVICE_NAME = "remote_machine_credentials"


'''
Debug:  Clear all user credentials

if os.path.exists(USERNAME_FILE):
    os.remove(USERNAME_FILE)


'''
# Function to load or prompt for both username and password
def load_or_prompt_credentials():
    if os.path.exists(USERNAME_FILE):
        with open(USERNAME_FILE, 'r') as f:
            username = f.read().strip()
            print(f"Loaded username {username} from {USERNAME_FILE}")
    else:
        username = input("Enter the username (format: DOMAIN\\USER): ")
        with open(USERNAME_FILE, 'w') as f:
            f.write(username)

    password = keyring.get_password(SERVICE_NAME, username)
    if password is None:
        password = getpass.getpass(prompt="Enter your password: ")
        keyring.set_password(SERVICE_NAME, username, password)
    
    return username, password

# Initially load the credentials
username, password = load_or_prompt_credentials()
print("Initial credentials loaded.")

# NOTE this is only medium security, and prevents you from exposing your username by sharing the script.  
# Usernames are stored in clear text, revise if greater security is desired


'''

### Function to Test PsExec Connection

This function aims to test the PsExec connection to a remote machine. If the first attempt to connect fails, it will prompt the user for credentials and try again.

'''
obfuscated_password = "********"

# Step 3: Decouple running command logic into a separate function
def run_psexec_command(remote_machine, username, password, command):
    testrun_psexec_command = f'{psexec_path} \\\\{remote_machine} -accepteula -u {username} -p {password} -s {command}'
    print(f"Executing command: {testrun_psexec_command.replace(password, obfuscated_password)}")
    
    try:
        subprocess.run(testrun_psexec_command, check=True)
        return True
    except subprocess.CalledProcessError as e:
        print(f"Command execution failed with error: {e}")
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False


# Function to test PsExec connection
def test_psexec_connection(remote_machine, username, password):
    
    
    # Initial Test
    if run_psexec_command(remote_machine, username, password, 'tasklist'):
        print(f"Successfully connected to {remote_machine}")
        return True
    
    # Step 5: Re-attempt logic
    for _ in range(2):  # Number of retries can be adjusted
        print(f"Failed to connect to {remote_machine}. Retrying...")
        
        # Delete Credentials and Re-prompt for both username and password
        keyring.delete_password(SERVICE_NAME, username)

        if os.path.exists(USERNAME_FILE):
            os.remove(USERNAME_FILE)
                        
        username, password = load_or_prompt_credentials()
        
        if run_psexec_command(remote_machine, username, password, 'dir'):
            print(f"Successfully connected to {remote_machine} on retry")
            return True
        else:
            print(f"Failed to connect to {remote_machine} on retry - Check Password or Permissions and Try Again")
            # Add failed machines to a list for later
            failed_machines.append(remote_machine)
        
    
    print(f"Retry failed for {remote_machine}. Please check your credentials or network connection.")
    return False

# Initialize list of failed machines
failed_machines = []


for folder in HECRAS_Deploy_Targets:
    if not folder.startswith("\\\\"):  # Check if folder does not start with \\
        continue  # Skip the rest of the loop for this folder
    
    remote_machine = folder.split('\\', 3)[2]
    print("remote_machine: ", remote_machine)
    print("")

    psexec_test_result = test_psexec_connection(remote_machine, username, password)

In [ ]:
# Delete Existing Files and Directories in the Target Folders

def clear_directory(dir_path):
    """Clears the directory given the path."""
    
    if not os.listdir(dir_path):
        print(f"No Files in the Target Folder ({dir_path}), No Folder Deletion Required")
        return

    for filename in os.listdir(dir_path):
        file_path = os.path.join(dir_path, filename)
        
        if not filename.startswith(Folder_Safety_Prefix):
            if os.path.isfile(file_path):
                print(f'Skipping file: {file_path} - Not matching the safety prefix.')
            elif os.path.isdir(file_path):
                print(f'Skipping directory: {file_path} - Not matching the safety prefix.')
            continue
        
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
            print(f'Successfully deleted: {file_path}')
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')

with ThreadPoolExecutor(max_workers=File_Copy_Threads) as executor:
    for remote_full in HECRAS_Deploy_Targets:
        executor.submit(clear_directory, remote_full)

In [ ]:
# Main RASCmdr-Deploy Script Logic

# Function to ignore files
def ignore_files(extensions):
    def _ignore_files(pathname, names):
        ignore = set()
        for extension in extensions:
            ignore.update({name for name in names if fnmatch.fnmatch(name, extension)})
        return ignore
    return _ignore_files


# Function to count files in a directory
def count_files_in_directory(directory):
    return sum([len(files) for _, _, files in os.walk(directory)])


# Function for each remote target folder's copy operation
def copy_directory_to_remote(src, dst, num_copies):
    try:
        total_copied_files = 0
        for i in range(1, num_copies + 1):
            new_folder = os.path.join(dst, os.path.basename(src) + '_' + str(i))
            print('\nStarting Copy:               ', new_folder)
            if os.path.exists(new_folder) and response.lower() == 'yes':
                shutil.rmtree(new_folder, ignore_errors=True)
            shutil.copytree(src, new_folder, ignore=ignore_files(exclusions), dirs_exist_ok=True)
            total_copied_files += count_files_in_directory(new_folder)
            time.sleep(1)
        
        return total_copied_files
    except Exception as e:
        print(f"Exception in copy_directory_to_remote for destination {dst}: {e}")
        return 0  # Return 0 so that we avoid the TypeError but still highlight an error


def copy_directory_to_remote_parallel(src, dsts, num_copies):
    if dsts:
        with ThreadPoolExecutor(max_workers=File_Copy_Threads) as executor:
            futures_to_dst = {executor.submit(copy_directory_to_remote, src, dst, num_copies): dst for dst in dsts}
            for future in as_completed(futures_to_dst):
                dst_for_this_future = futures_to_dst[future]
                print("")
                print(f"Finished Deploying:           {dst_for_this_future} with {Number_Parallel_Runs} Parallel Runs {future.result()} files copied.")
        print("\nAll RAS Project Folders are Deployed.")
    else:
        print("No destination directories provided.")

# Call Function to copy from HECRAS_Project_To_Run to HECRAS_Deploy_Targets
copy_directory_to_remote_parallel(HECRAS_Project_To_Run, HECRAS_Deploy_Targets, Number_Parallel_Runs)

In [ ]:
# Create Batch Files in the Target Directories

# Function to create batch files in the Target Directories
def RAS_Cmdr(hecras_project_directory):
  
    # Scanning the RAS Project Directory for .rasmap file to infer project name
    # Since .prj files are also used by shapefiles, we use a file extension unique to HEC-RAS
    for file_name in os.listdir(hecras_project_directory):
        # Check if the file has .rasmap extension
        if file_name.endswith('.rasmap'):
            # Return the file name without the .rasmap extension
            hecrasproject_prj_filename = file_name[:-7] 
            # Print Message if no .rasmap file is found
            print("")
            print("HEC-RAS Project Name: " + hecrasproject_prj_filename)
            print("")
 
    # Convert Remote Paths for local execution using Remote_File_Share_Path
    if hecras_project_directory.startswith('\\\\'):
        hecras_project_directory_absolute = Remote_File_Share_Path + hecras_project_directory[hecras_project_directory.find('\\', 2):]
        # print("DEBUG Remote_File_Share_Path: ", Remote_File_Share_Path)
        print(hecras_project_directory[hecras_project_directory.find('\\', 2):])
        print("")
        print("*****  HEC-RAS Path is in a remote file share.  *****")
        print("Converted :" + hecras_project_directory)
        print ("       to: " + hecras_project_directory_absolute)
        
    else:
        hecras_project_directory_absolute = hecras_project_directory
    

    # Target HEC-RAS Project File Path (For script execution)
    RAS_Cmdr_proj_filename = os.path.join(hecras_project_directory, hecras_project_name + '.prj')

    # Target HEC-RAS Project File Path (Absolute path for writing the batch file)
    RAS_Cmdr_proj_filename_absolute = os.path.join(hecras_project_directory_absolute, hecras_project_name + '.prj')


    # Check if the project file exists
    if os.path.exists(RAS_Cmdr_proj_filename):
        print("")
        print(f"Reading Project File at {RAS_Cmdr_proj_filename}")
    else:
        print("")
        print(f"Project file not found at {RAS_Cmdr_proj_filename}")
        return


    # Read the lines of the project file and get the plan file numbers
    with open(RAS_Cmdr_proj_filename, 'r') as f:
        lines = f.readlines()
        plan_lines = [line.strip().split('=')[1] for line in lines if line.startswith('Plan File=')]

    # Create a DataFrame to display the plan files
    df = pd.DataFrame(plan_lines, columns=['Plan Files'])

    print("The PRJ file contains the following plans:")
    print("")
    print(df)
    print("")
    print("Now Batch Files will be created for each plan file.")
    print("")

    # Create batch files for each plan file
    for plan_line in plan_lines:
        
        #Increment the plan number
        plan_number = plan_line.split('=')[-1] 

        print ("------------------------------------")
        print ("Current plan number: " + plan_number)
        

        
        # Assemble the full HEC-RAS Batch File Command
        # RAS_Cmdr_proj_filename is the same for all plan files, so it is outside the loop
        RAS_Cmdr_plan_filename = os.path.join(hecras_project_directory, hecras_project_name + '.' + plan_number)
        RAS_Cmdr_plan_filename_absolute = os.path.join(hecras_project_directory_absolute, hecras_project_name + '.' + plan_number)
        # Check if the plan file exists
        if not os.path.exists(RAS_Cmdr_plan_filename):
            # Notify the user if the plan file is not found
            print(f"Plan file {plan_number} not found at {RAS_Cmdr_plan_filename}")
            continue

        # Print the paths for debugging
        print ("HEC-RAS Executable Path: ", hecras_exe_path)
        print ("Current Project File Path: ", RAS_Cmdr_proj_filename)
        print ("Current Plan File Path:    ", RAS_Cmdr_plan_filename)
        
        # Name Batch File per HEC-RAS Project Name and Plan Number
        RAS_Cmdr_bat_file_name = hecras_project_name + '.' + plan_number + '.run.bat'
        print ("RAS_Cmdr_bat_file_name: ", RAS_Cmdr_bat_file_name)

        # Assemble the full HEC-RAS Batch File Path (for writing)
        RAS_Cmdr_bat_file_path = os.path.join(hecras_project_directory, RAS_Cmdr_bat_file_name)

        # Assemble the full HEC-RAS Batch File Command
        RAS_Cmdr_bat_file_command = f'"{hecras_exe_path}" -c "{RAS_Cmdr_proj_filename_absolute}" "{RAS_Cmdr_plan_filename_absolute}"'
        # THIS COMMAND CAN BE RUN WITH SUBPROCESS.CALL USING PSEXEC TO RUN ON REMOTE MACHINE
        print(f"BAT File Command: {RAS_Cmdr_bat_file_command}")
     
        # Write the Batch file for this plan file
        with open(RAS_Cmdr_bat_file_path, 'w') as f:
            f.write(RAS_Cmdr_bat_file_command)
        print("")
        print(f"Successfully created batch file at {RAS_Cmdr_bat_file_path}")
        print("")

# Iterate over HECRAS_Deploy_Targets.  For each subdirectory of HECRAS_Deploy_Targets, call RAS_Cmdr
for remote_full in HECRAS_Deploy_Targets:
    for remote_sub in os.listdir(remote_full):
        print ("------------------ RAS Commander: Command Line is All You Need ------------------")
        print ("")
        print ("RAS_Cmdr called for: ", os.path.join(remote_full, remote_sub))
        RAS_Cmdr(os.path.join(remote_full, remote_sub))
        print ("RAS_Cmdr Completed Building BAT FIles for: ", os.path.join(remote_full, remote_sub))
        print ("")
        print ("")
        print ("")

In [ ]:
# Remote Execution of HEC-RAS Batch Files

failed_files = []
running_threads = []

def execute_bat(queue, folder, folder_lock):
    while True:
        try:
            bat_file = queue.get(block=False)
        except Empty:
            break

        # Define the path to the batch file
        bat_path = os.path.join(folder, bat_file)
        print("")

        # Workflow if folder is on remote machine   
        if folder.startswith('\\\\'):
            
            # Replace from "\\" to the next "\" with the Remote_File_Share_Path
            bat_path = Remote_File_Share_Path + folder[folder.find('\\', 2):] + "\\" + bat_file
            

            # For remote paths, grab remote machine 
            # print("Folder: ", folder)
            remote_machine = folder.split('\\', 3)[2]
            print("remote_machine: ", remote_machine)

            # This is the remote command that will be executed with PSEXEC
            if Psexec_Run_In_System_Account == "Yes":
                # For the first condition (running as System account without interaction)
                ppsexec_cmd = f'{psexec_path} \\\\{remote_machine} -u {username} -p {password} -s -{Psexec_Priority} powershell -windowstyle hidden -command "& \'{bat_path}\'"'
            else: 
                # If not "Yes, run interactive with the specified Session ID"
                psexec_cmd = f'{psexec_path} \\\\{remote_machine} -u {username} -p {password} -i {Psexec_Session_ID} -{Psexec_Priority} powershell -windowstyle hidden -command "& \'{bat_path}\'"'
            
            obfuscated_password = "********"
        
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
            # Print the command with the obfuscated password by replacing it only for the print statement
            print(f"[{timestamp}] Executing command: {psexec_cmd.replace(password, obfuscated_password)}")

            with folder_lock:  # Ensure that only one thread can access this subfolder at a time
                try:
                    print(f"[{timestamp}] Running {bat_file} in {folder}")
                    subprocess.check_call(psexec_cmd)
                    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Update timestamp after command execution
                    print(f"[{timestamp}] Finished {bat_file} in {folder}")
                    
                    # Copy DSS output files after thread execution (Deprecated)
                    # find_and_copy_files(folder, dss_archive_directory, dss_working_directory, dss_file_prefix)
                    
                    print("  ")
                except subprocess.CalledProcessError as e:
                    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Update timestamp if an error occurs
                    print(f"[{timestamp}] Failed to run {bat_file} in {folder}. Error: {str(e)}")
                    failed_files.append(bat_file)
        else:
            print("Local Execution")
            try:
                timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                print(f"[{timestamp}] Running {bat_file} locally in {folder}")
                subprocess.call(bat_path)  # Execute the .bat file locally
                timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Update timestamp after command execution
                print(f"[{timestamp}] Finished {bat_file} locally in {folder}")
                print("  ")
            except Exception as e:
                timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Update timestamp if an error occurs
                print(f"[{timestamp}] Failed to run {bat_file} locally in {folder}. Error: {str(e)}")
                failed_files.append(bat_file)
      

# Assuming Remote_File_Share_Path is defined somewhere outside the function in the global scope
# Remote_File_Share_Path = "..."


def get_remote_subfolders(target_folders):
    """
    This function returns a list of subfolders for the given remote folders. 
    If a remote subfolder's path starts with '\\', it corrects the path 
    using the globally defined Remote_File_Share_Path.

    Parameters:
    - target_folders (list): A list of paths to the remote folders.

    Returns:
    - list: A list of corrected subfolder paths.
    """
    print(f"target_folders: {target_folders}")
    # Initialize an empty list to store the remote subfolders
    remote_subfolders = []
    
    # Loop through each remote folder in the target folders list
    for remote_folder in target_folders:
        try:
            # Generate a list of all subfolders for the current remote folder
            subfolders = [
                os.path.join(remote_folder, dir)  # Join the remote folder path with the directory name
                for dir in os.listdir(remote_folder)  # List all items in the current remote folder
                if os.path.isdir(os.path.join(remote_folder, dir))  # Check if the item is a directory (subfolder)
            ]
            
            # Append the found subfolders to the remote_subfolders list
            remote_subfolders.extend(subfolders)
            
        except Exception as e:
            # Print any errors encountered when accessing the remote folders
            print(f"Error accessing {remote_folder}: {e}")
            
    # Return the list of corrected subfolder paths
    return remote_subfolders

# Initialize an empty list to store the remote subfolders
remote_subfolders = []

# Get the remote subfolders
remote_subfolders = get_remote_subfolders(HECRAS_Deploy_Targets)
print(f"remote_subfolders: {remote_subfolders}")
print("")

# Distributes the files among remote machines
def distribute_bat_files_among_remote_machines(missing_bat_files, remote_subfolders):
    if not remote_subfolders:
        print("No remote subfolders provided.")
        return []

    queues = [Queue() for _ in remote_subfolders]
    for i, bat_file in enumerate(missing_bat_files):
        queues[i % len(remote_subfolders)].put(bat_file)
        print(f"Added {bat_file} to queue {i % len(remote_subfolders)}")
    return queues


# Runs a single thread for each queue
def run_remote_threads(queues, execution_subfolders, folder_locks):
    running_threads = []
    for i, (queue, folder, folder_lock) in enumerate(zip(queues, execution_subfolders, folder_locks)):
        print(f"Starting thread {i} using {folder}")
        thread = threading.Thread(target=execute_bat, args=(queue, folder, folder_lock), daemon=True)
        thread.start()
        running_threads.append(thread)
        time.sleep(5)  # Pause for 5 seconds before starting the next thread
    return running_threads


#Main Execution Script Logic

first_target = HECRAS_Deploy_Targets[0]
first_subfolder = next(os.walk(first_target))[1][0]
first_subfolder_path = os.path.join(first_target, first_subfolder)

run_plan_title_from_pxx_files = [f for f in os.listdir(first_subfolder_path) if f.endswith('.run.bat')]

print(f"run_plan_title_from_pxx_files: {run_plan_title_from_pxx_files}")
print("")
# bat_files_for_run_plans = run_plan_title_from_pxx_files
bat_files_for_run_plans = [file for file in run_plan_title_from_pxx_files]
print(f"bat_files_for_run_plans: {bat_files_for_run_plans}")
print("")
# Build missing_bat_files_df from bat_files_for_run_plans and run_plan_title_from_pxx_files
missing_bat_files_df = pd.DataFrame({"PXX File": run_plan_title_from_pxx_files, "Missing BAT File": bat_files_for_run_plans},)
print("")
print(f"missing_bat_files_df: {missing_bat_files_df}")
print("")
print("\nMissing BAT files:")
print("")
print(missing_bat_files_df)


# Define queues for each remote machine
# Distribute the missing bat files among the queues
queues = distribute_bat_files_among_remote_machines(missing_bat_files_df["Missing BAT File"], remote_subfolders)
print(f"queues: {queues}")

# Create a lock for each subfolder
print(f"queues: {queues}")  
folder_locks = [threading.Lock() for _ in remote_subfolders]

# execution_subfolders is a copy of subfolders, which will be modified during execution
execution_subfolders = remote_subfolders.copy()

# Initialize and start multiple threads based on the number of remote machines

# Loop ensures that the main program waits for each thread in running_threads to complete its execution before proceeding
running_threads = run_remote_threads(queues, execution_subfolders, folder_locks)
for thread in running_threads:
    thread.join()

# Notify the user that all .bat files have been executed
print("All .bat files executed.")

# Print the list of files that failed to run
if failed_files:
    print("The following files failed to run:")
    for file in failed_files:
        print(file)

In [ ]:
# Copy all matching files from the list of directories to a single directory, replace if larger

# Additional/Optional Settings for Results Collection and INCLUDE/EXCLUDE Filters

# DSS Search Filter Word - useful for filtering out files that are not needed.  Only applies to DSS files.
# NEED TO REVISE FUNCTIONALITY PER ABOVE
DSS_Search_Filter_Include = ""
print("DSS_Search_Filter_Include: " + DSS_Search_Filter_Include)

# Exclude Search Term (e.g. "HMS" to prevent HMS DSS files from being copied at this step)
DSS_Search_Filter_Exclude = ""
print("DSS_Search_Filter_Exclude: " + DSS_Search_Filter_Exclude)

# File Extensions to Include in Search.  Numbered extensions should be placed in second half of list.
file_extensions = ["dss", "hdf", "rst"] + [f"{prefix}{i:02}" for prefix in ["b", "c", "x", "r", "bco"] for i in range(1, 100)]
print("File Extensions: " + str(file_extensions))

### HEC-RAS File Extensions and Descriptions:
### A quick primer on file types that you may be interested in collecting
# 1. **.dss: HEC-RAS DSS output file.
# 2. **.rst: RAS restart file
# 3. **.bXX: Boundary Conditions ASCII input file
# 4. **.cXX: Geometry Preprocessor Output Binary
# 5. **.xXX: Geometry ASCII input file
# 6. **.IC.OXX: Initial Condition binary
# 7. **.gXX.hdf: Geometry in HDF format
# 8. **.pXX.hdf: RAS results in HDF format
# 9. **.tmp.hdf: Temp File with Incomplete RAS results in HDF format
# An example with all file types is shown below:
# file_extensions = ["dss", "hdf", "rst"] + [f"{prefix}{i:02}" for prefix in ["b", "c", "x", "r", "O", "IC.O", "bco"] for i in range(1, 100)]

# NOTE: **.tmp.hdf files are excluded in the code below because they are not complete results files.

# Function to copy a single file, replace if it's newer
def copy_file(source_file, destination_file):
    try:
        if os.path.exists(destination_file):
            if os.path.getmtime(source_file) > os.path.getmtime(destination_file):
                shutil.copy2(source_file, destination_file)
                print(f"Copied {source_file} to {destination_file} (replaced due to being newer).")
                return True
            else:
                dummyvar = 1 #prevents error if debug line is commented out
                #print(f"Skipped {source_file} as the destination file is newer or the same age.")
        else:
            shutil.copy2(source_file, destination_file)
            print(f"Copied {source_file} to {destination_file}.")
            return True
    except PermissionError:
        print(f"Permission denied when attempting to copy {source_file}. Moving to next file.")
    return False

# Check if HECRAS_Project_To_Run is a file
if os.path.isfile(HECRAS_Project_To_Run):
    print(f"Error: {HECRAS_Project_To_Run} is a file. Please delete the file or choose a different output directory.")
else:
    # Create the output directory if it doesn't exist
    if not os.path.exists(HECRAS_Project_To_Run):
        os.makedirs(HECRAS_Project_To_Run)
        print(f"Created output directory: {HECRAS_Project_To_Run}")

    files_copied = 0

    for source_directory in remote_subfolders:
        print(f"Processing source directory: {source_directory}")
        
        if os.path.basename(source_directory).startswith(Folder_Safety_Prefix):
            for root, dirs, files in os.walk(source_directory):
                # Exclude files with "terrain" in the path
                if "terrain" not in root:
                    for file in files:
                        # .tmp.hdf files are temporary files created by HEC-RAS, and usually indicate that the model is still running or failed to run
                        # We explicitly exclude these files from the copy process
                        if any(re.search(fr"\.{extension}$", file) and DSS_Search_Filter_Include in file for extension in file_extensions) and not file.endswith('.tmp.hdf'):
                            source_file = os.path.join(root, file)
                            destination_file = os.path.join(HECRAS_Project_To_Run, file)
                            if copy_file(source_file, destination_file):
                                files_copied += 1
                        else:
                            dummyvar = 1 #prevents error if debug line is commented out
                            # print(f"Skipped {file} due to not matching the search criteria.")
                else:
                    print(f"Skipped directory {root} due to containing 'terrain'.")
        else:
            print(f"Skipped source directory {source_directory} due to not matching safety prefix.")

    if files_copied == 0:
        print("Error: No files found to copy.")
    else:
        print(f"Successfully copied {files_copied} files.")

In [ ]:
# Cleanup: delete all deployed folders from remote machines using safety prefix

# Delete Existing Files and Directories in the Target Folders

# User Confirmation before Deleting Folders
confirmation = input("Are you sure you want to delete all remote deployed folders? Type 'yes' to confirm: ")

if confirmation.lower() not in ['yes', 'no']:
    print("Invalid input. Please type 'yes' or 'no'.")
elif confirmation.lower() == 'no':
    print("Aborting deletion process.")
else:
    # Function to clear directories with safety prefix logic
    def clear_directory(dir_path):
        """Clears the directory given the path."""
        
        if not os.listdir(dir_path):
            print(f"No Files in the Target Folder ({dir_path}), No Folder Deletion Required")
            return

        for filename in os.listdir(dir_path):
            file_path = os.path.join(dir_path, filename)
            
            if not filename.startswith(Folder_Safety_Prefix):
                if os.path.isfile(file_path):
                    print(f'Skipping file: {file_path} - Not matching the safety prefix.')
                elif os.path.isdir(file_path):
                    print(f'Skipping directory: {file_path} - Not matching the safety prefix.')
                continue
            
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
                print(f'Successfully deleted: {file_path}')
            except Exception as e:
                print(f'Failed to delete {file_path}. Reason: {e}')

    with ThreadPoolExecutor(max_workers=File_Copy_Threads) as executor:
        for remote_full in HECRAS_Deploy_Targets:
            executor.submit(clear_directory, remote_full)